In [2]:
import tabula
import pandas as pd
import numpy as np

In [3]:
gender_df = tabula.read_pdf("Perm-FT-Faculty-and-Post-Doc-Fellows-X-Gender-X-Tenure-Status-Fall-2010-2020_20210406.pdf", pages='all')


In [4]:
gender_df = gender_df[0]

In [5]:
to_drop = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","Unnamed: 1", "Unnamed: 2", "Unnamed: 4","Unnamed: 6",
          "Unnamed: 8", "Unnamed: 10", "Unnamed: 12", "Unnamed: 14", "Unnamed: 16", "Unnamed: 18", "Unnamed: 20"]

In [6]:
# dropping the columns where I don't care about them
gender_df.drop(to_drop, inplace=True, axis=1)


In [7]:
# list comprehension to make the list of new year labels
new_cols = [str(i) for i in range(2010,2021, 1)]   

In [8]:
# giving the old names the current column names
old_names = gender_df.columns

# adding the first col (b/c we're not changing it)
new_names = [gender_df.columns[0]] +  new_cols

In [9]:
# setting the columns to the new names
gender_df.columns = new_names

In [10]:
# dropping the first row because there's not much point in it
gender_df = gender_df.drop(labels=0, axis=0)

In [11]:
(1%2 != 0)

True

In [13]:
# https://stackoverflow.com/questions/49162195/insert-value-based-on-row-index-number-in-a-pandas-dataframe
# setting up various conditions to search in np.select with
tenure_status = [(gender_df.index > 0) & (gender_df.index <=4),
               (gender_df.index > 4) & (gender_df.index <=8),
               (gender_df.index > 8) & (gender_df.index <=12),
               (gender_df.index > 12) & (gender_df.index <=16),
               (gender_df.index > 16) & (gender_df.index <=20),
               (gender_df.index > 20) & (gender_df.index <=25)]           

# setting the indexes we'll use to label here
# choosing the rows which are NA
ten_indexes = gender_df[gender_df["2010"].isna()]["Tenure"]

# changing the values in Tenure to describe the groups
gender_df["Tenure"] = np.select(tenure_status, ten_indexes)


# odd numbers are male always
# even numbers are
# 

# setting the conditions for the genders and all
genders = [(gender_df.index%2 == 0) & (gender_df.index%4 != 0),
            (gender_df.index%2 != 0),
            (gender_df.index%4 == 0)]

gend_index = ["Female","Male","Total"]

# changing the values in Tenure to describe the groups
gender_df["Gender"] = np.select(genders, gend_index)

# dropping nan rows
gender_df = gender_df.dropna()

# going ahead and setting tenure as an index
gender_df = gender_df.set_index([gender_df["Tenure"],gender_df["Gender"]])

# dropping the tenure row now that it's an index
gender_df = gender_df.drop(["Tenure","Gender"], axis=1)



In [17]:
gender_df

2010   2011   2012   2013   2014   2015  \
Tenure                  Gender                                             
Tenured                 Female    418    420    431    430    440    480   
                        Male      978    961    949    940    919    955   
                        Total   1,396  1,381  1,380  1,370  1,359  1,435   
Tenure Track            Female    181    195    194    202    207    225   
                        Male      255    254    263    259    264    248   
                        Total     436    449    457    461    471    473   
Tenured and TenureTrack Female    599    615    625    632    647    705   
                        Male    1,233  1,215  1,212  1,199  1,183  1,203   
                        Total   1,832  1,830  1,837  1,831  1,830  1,908   
Fixed Term              Female    841    876    873    942    934  1,054   
                        Male      626    648    645    679    696    797   
                        Total   1,467  1,524  1,518  1,621  1,630  1,851   
All                     Female  1,440  1,491  1,498  1,574  1,581  1,759   
                        Male    1,859  1,863  1,857  1,878  1,879  2,000   
                        Total   3,299  3,354  3,355  3,452  3,460  3,759   
Post Doctoral Fellows   Female    386    367    367    357    364    382   
                        Male      478    468    440    440    422    447   
                        Total     864    835    807    797    786    829   

                                 2016   2017   2018   2019   2020  
Tenure                  Gender                                     
Tenured                 Female    488    505    513    524    536  
                        Male      932    927    921    896    891  
                        Total   1,420  1,432  1,434  1,420  1,427  
Tenure Track            Female    229    214    210    217    228  
                        Male      241    225    211    210    202  
                        Total     470    439    421    427    430  
Tenured and TenureTrack Female    717    719    723    741    764  
                        Male    1,173  1,152  1,132  1,106  1,093  
                        Total   1,890  1,871  1,855  1,847  1,857  
Fixed Term              Female  1,121  1,177  1,209  1,265  1,313  
                        Male      839    839    888    929    915  
                        Total   1,960  2,016  2,097  2,194  2,228  
All                     Female  1,838  1,896  1,932  2,006  2,077  
                        Male    2,012  1,991  2,020  2,035  2,008  
                        Total   3,850  3,887  3,952  4,041  4,085  
Post Doctoral Fellows   Female    387    388    357    368    370  
                        Male      462    424    405    417    401  
                        Total     849    812    762    785    771

In [15]:
# writing the data to a json
gender_df.to_csv(r"faculty_gender.csv")